## The N-Body Problem:

The N-Body problem is the problem of predicting the individual motions of a group of objects in space interacting with
each other gravitationally.  The informal definition, according to Astrophysicist Reinhardt Rosenberg is "Given the quasi-
steady orbital properties of a group of celestial bodies--their instantaneous position, velocity, and time--predict their
interactive forces; and consequently, predict their true orbital motions for any or all future times."

This problem has existed for a long time - the two-body problem was solved by Johann Bernoulli in the early 18th Century,
and the three-body problem was solved for a Choreography (in which all the bodies move on the same curve, and collisions
do not need to be taken into acount) by Joseph-Louis Lagrange in 1772.  by Charles-Eugene Delaunay in the mid 19th
century.  Solutions for larger configurations have been largely relegated to the realm of computer modeling.

The two-body problem is a classic example of differential equation solving differential equations.

The general solution to the n-body problem considers the mass, position and acceleration of n separate bodies, and the
effect of the mutual gravitational attraction between any pair of them, over all possible subsets of pairs.  In other
words, to calculate a three body problem (sun, Earth and moon, for example), the gravitational attraction of the sun
to the Earth and of the sun to the moon must be calculated; so too must the attraction of the moon to the sun and the
moon to the Earth; and finally the Earth to the sun and the Earth to the moon.  In order to plot an orbit using these
differential calculations, these calculations--the differential equation series known as the n-body equations of motion--
are calculated repeatedly based on the results of the previous calculation.

This is an ODE model, requiring a precise set of initial values and with a great deal of sensitivity to the initial input.
As there was not a readily available set of data for the Gliese system (as there was in the other simulation), I made
the decision to create some fake planets with fake (but plausible) data, in the hopes of finding an unstable orbit that
would show just how input sensitive the model is.  In the end, I was unable to create a truly unstable orbit (in either
direction - decay or ejection), but I did create a four body system with an extremely eccentric trio of orbits.

In [ ]:
import math
import matplotlib.pyplot as plt

## The Classes:

The Point Class is simply a set of coordinates in 3d space.  It was implemented for efficiency of retrieval, so that I
didn't have to mess around with zipping three lists together in a meaningful way after pulling them out of a dictionary.
The Class also helps uniformity by allowing position and Velocity to be implemented without the need for additional
classes or finesse.

The Body Class defines a body in space; position and velocity are both Points (as alluded to directly above).  Position
and mass are both floats in scientific notation (handled easily by Python with the math library imported).  Velocity is
initialized as a rough average in meters per second for each planet, not taking into account whether the orbit is
beginning at its perhelion (the point in a body's orbit at which it is closest to the body around which it orbits) or
its aphelion (the point at which a body is farthest from the body around which it orbits), or else somewhere in between.

Name and color should be self-explanatory (though it should be noted that color uses the set of verbally defined CSS
colors out of convenience and not out of a thirst for accuracy -- the color of Saturn should not be described in
casual conversation with astronomers or astrophysicists as 'lemon chiffon').

In [ ]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Body:
    def __init__(self, position, mass, velocity, name="", color=""):
        self.position = position
        self.mass = mass
        self.velocity = velocity
        self.name = name
        self.color = color

## The Bodies:

One star, eight familiar planets and one dwarf planet.  The value types are noted above.  They're saved here as
dictionaries for ease of access by other functions, and ease of use in defining n-sets of bodies for calculation.
The values for position (for the planets) and their velocities are adapted from values taken from solarsystem.nasa.gov
and wikipedia.org.  Note that the planets are fairly arbitrarily graphed in a 2d plane at the start.  This is supremely
unlikely, but much easier to deal with when setting initial values for them.

I had hoped to include values for Gliese, but the information on exact velocities was unavailable without more math than
I was able to do on short notice.  I may add that data in the future, for an open-source release.

Since these values model an existing system, they tend toward stable orbits even in circumstances where some planets are
removed from consideration (unless you change some values for Jupiter and move it closer to any of the other planets).
To create a more interesting, less stable system, I created key:value sets for a fictitious system, composed of a star
and three planets, with names taken from works of science fiction.  The star in this system is modeled as a red giant
with a mass roughly twice that of our own sun; its innermost planet is the same distance from the sun as Mars is from
our own sun, with a mass slightly greater than that of Venus.  Its outermost planet is roughly the mass of Neptune, and
is at what would be, in our own system, roughly the halfway point between Mars and Jupiter.  Between those two planets
is a planet with the exact mass of Venus.

In [ ]:
sun = {"position": Point(0, 0, 0), "mass": 2e30, "velocity": Point(0, 0, 0)}
mercury = {"position": Point(0, 5.7e10, 0), "mass": 3.285e23, "velocity": Point(47000, 0, 0)}
venus = {"position": Point(0, 1.1e11, 0), "mass": 4.8e24, "velocity": Point(35000, 0, 0)}
earth = {"position": Point(0, 1.5e11, 0), "mass": 6e24, "velocity": Point(30000, 0, 0)}
mars = {"position": Point(0, 2.2e11, 0), "mass": 2.4e24, "velocity": Point(24000, 0, 0)}
jupiter = {"position": Point(0, 7.7e11, 0), "mass": 1e28, "velocity": Point(13000, 0, 0)}
saturn = {"position": Point(0, 1.4e12, 0), "mass": 5.7e26, "velocity": Point(9000, 0, 0)}
uranus = {"position": Point(0, 2.8e12, 0), "mass": 8.7e25, "velocity": Point(6835, 0, 0)}
neptune = {"position": Point(0, 4.5e12, 0), "mass": 1e26, "velocity": Point(5477, 0, 0)}
pluto = {"position": Point(0, 3.7e12, 0), "mass": 1.3e22, "velocity": Point(4748, 0, 0)}

rao = {"position": Point(0, 0, 0), "mass": 4.1e30, "velocity": Point(0, 0, 0)}
arrakis = {"position": Point(0, 2.2e11, 0), "mass": 5.1e23, "velocity": Point(30000, 0, 0)}
risa = {"position": Point(0, 3.4e11, 0), "mass": 4.8e23, "velocity": Point(10100, 0, 0)}
ego = {"position": Point(0, 4.9e11, 0), "mass": 1e26, "velocity": Point(6835, 0, 0)}

## The math

I won't go into great detail on the math - I am neither a mathematician or a physicist.  But I will outline what is
going on in most of the functions which go into the calculation.  Normally to solve a differential equation, I would
just resort to .odeint() in numpy; clearly that wasn't going to be an option here, so I did it myself.

Compute_sba - computes the acceleration of a single body; for each body, it takes the square difference between that
body's x position and the x position of every other body in the system, then does the same for y and z.  It takes the
square root of the sums of those squares in order to calculate a radius containing half the mass of the system (its
characteristic size). It then multiplies the gravitational constant by the mass and divides that product by the
characteristic size.  It multiplies the result of this value by the difference between the other body's x position and
the current body's y position, then adds that to its acceleration.  Then repeats this process for their y and z
positional values.

Update_velocity is simply a container for calling compute_sba, taking the returned acceleration calculated for one body
through examination of all the other bodies in the system, and altering the velocity of the current body accordingly.

In [ ]:
def compute_sba(bodies, body_index):
    grav = 6.67408e-11
    acc = Point(0, 0, 0)
    current_body = bodies[body_index]

    for index, other in enumerate(bodies):
        if index != body_index:
            r = (current_body.position.x - other.position.x) ** 2 + (
                        current_body.position.y - other.position.y) ** 2 + (
                            current_body.position.z - other.position.z) ** 2
            r = math.sqrt(r)
            tmp = grav * other.mass / r ** 3
            acc.x += tmp * (other.position.x - current_body.position.x)
            acc.y += tmp * (other.position.y - current_body.position.y)
            acc.z += tmp * (other.position.z - current_body.position.z)

    return acc


def update_velocity(bodies, time_step=1):
    for index, current_body in enumerate(bodies):
        acceleration = compute_sba(bodies, index)

        current_body.velocity.x += acceleration.x * time_step
        current_body.velocity.y += acceleration.y * time_step
        current_body.velocity.z += acceleration.z * time_step

Update position multiplies the velocity for each dimension by the time step size (default of 1, typically set to 100 so
it doesn't take forever to run the program), then adds it to the appropriate dimension in the position.

Grav step is simply a container which calls both update velocity and update position.  Werner Heisenberg would be
appalled at the thought that we know how fast these bodies are moving _and_ where they are simultaneously.

In [ ]:
def update_position(bodies, time_step=1):
    for body in bodies:
        body.position.x += body.velocity.x * time_step
        body.position.y += body.velocity.y * time_step
        body.position.z += body.velocity.z * time_step


def grav_step(bodies, time_step=1):
    update_velocity(bodies, time_step=time_step)
    update_position(bodies, time_step=time_step)

Plot_orbits is a reasonably basic 3d plot in using matplotlib.

Run_simulation creates a list of lists, orbital_history, which creates a dictionary for each body involved in the
problem.  The dictionary consists of lists of x, y, and z coordinates, as well as the body's name (and it's color, for
easy access by plot_orbits, to ensure that every planet has a color that matches its real color reasonably closely.
Except for Saturn).

In [ ]:
def plot_orbits(bodies):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    max_range = 0

    for current_body in bodies:
        max_dim = max(max(current_body["x"]), max(current_body["y"]), max(current_body["z"]))
        if max_dim > max_range:
            max_range = max_dim
        ax.plot(current_body["x"], current_body["y"], current_body["z"], c=current_body["color"],
                label=current_body["name"])

    ax.set_xlim([-max_range, max_range])
    ax.set_ylim([-max_range, max_range])
    ax.set_zlim([-max_range, max_range])
    ax.legend()
    plt.show()

In [ ]:
def run_simulation(bodies, names=None, time_step=1, number_of_steps=2000000, report_freq=100):
    orbital_history = []

    for current_body in bodies:
        orbital_history.append({"x": [], "y": [], "z": [], "name": current_body.name, "color": current_body.color})

    for i in range(1, number_of_steps):
        grav_step(bodies, time_step=1000)

        if i % report_freq == 0:
            for index, position in enumerate(orbital_history):
                position["x"].append(bodies[index].position.x)
                position["y"].append(bodies[index].position.y)
                position["z"].append(bodies[index].position.z)

    return orbital_history

## Experiment 1 -

Due to egregiously long computation times (this is for sure a project that would benefit from parallel computing - each
core could be crunching a separate body relation or set of body relations at each time step), I have yet to get a
complete graph for a 10-body problem including all 8 planets and pluto, as well as the Sun.  I have divided up the
available bodies into some reasonable groupings in order to limit the number of bodies under consideration to five.
Alas, poor Pluto isn't in any of the sets.

This first set of bodies is the Sun and all of the gas giants in the Solar System (Jupiter, Saturn, Uranus and Neptune).

Please note in all of these examples that the sun is stationary.  As it is stationary, it is not plotted by matplotlib.
I hope to fix this issue in the future, but did not have time to figure out how to make it perform a plottable set of
motions without adversely affecting the rest of the bodies in the calculation.  Intuitively, in the case of simulations
which generate a stable set of elliptical orbits, the sun is at their center.  In the case of unstable orbits... it's
a little more difficult to deduce that.

In [ ]:
bodies = [
    Body(position=sun["position"], mass=sun["mass"], velocity=sun["velocity"], name="Sun", color="gold"),
    Body(position=jupiter["position"], mass=jupiter["mass"], velocity=jupiter["velocity"], name="Jupiter",
         color="orange"),
    Body(position=saturn["position"], mass=saturn["mass"], velocity=saturn["velocity"], name="Saturn",
         color="lemonchiffon"),
    Body(position=neptune["position"], mass=neptune["mass"], velocity=neptune["velocity"], name="Neptune",
         color="steelblue"),
    Body(position=uranus["position"], mass=uranus["mass"], velocity=uranus["velocity"], name="Uranus",
         color="mediumturquoise")
]

motions = run_simulation(bodies, time_step=100, number_of_steps=2000000, report_freq=1000)
plot_orbits(motions)

This second set of bodies consists of just the innermost planets in the solar system (Mercury, Venus, the Earth, and
Mars), as well as the Sun.

In [ ]:
bodies = [
    Body(position=sun["position"], mass=sun["mass"], velocity=sun["velocity"], name="Sun", color="gold"),
    Body(position=mercury["position"], mass=mercury["mass"], velocity=mercury["velocity"], name="Mercury",
         color="burlywood"),
    Body(position=venus["position"], mass=venus["mass"], velocity=venus["velocity"], name="Venus",
         color="magenta"),
    Body(position=earth["position"], mass=earth["mass"], velocity=earth["velocity"], name="Earth",
         color="mediumblue"),
    Body(position=mars["position"], mass=mars["mass"], velocity=mars["velocity"], name="Mars",
         color="firebrick")
]
motions = run_simulation(bodies, time_step=100, number_of_steps=2000000, report_freq=1000)
plot_orbits(motions)

This final set was specifically designed (through trial and error) to use reasonable values for the bodies in its set,
but to produce a result that was an unstable orbit of some kind.  The two varieties of unstable orbit are decaying,
in which a body's orbit eventually causes it to fall into the parent body around which it is orbiting, or else ejection,
in which a body is ejected parabolically from its orbit, typically through interaction with a much more massive object.

Unfortunately, even after a large number of attempts with various parameters, I was unable to create systems which
exhibited either of these behaviors within a reasonable number of time steps (2 million).  I was able to create a
system in which the outermost planet has pulled its neighbor into a short orbit with a roughly 30 degree angle.  Given
enough time, I suspect that the result might be either eventual ejection, or else capture of the smaller body by the
much larger one as a moon.  However, that did not occur within the first 2 million steps, and I did not have time to
keep the simulation running indefinitely in the hopes of observing a different outcome.

In [ ]:
bodies = [
    Body(position=rao["position"], mass=rao["mass"], velocity=rao["velocity"], name="Rao", color="red"),
    Body(position=arrakis["position"], mass=arrakis["mass"], velocity=arrakis["velocity"], name="Arrakis",
         color="khaki"),
    Body(position=risa["position"], mass=risa["mass"], velocity=risa["velocity"], name="Risa",
         color="magenta"),
    Body(position=ego["position"], mass=ego["mass"], velocity=ego["velocity"], name="Ego", color="teal")
]

motions = run_simulation(bodies, time_step=100, number_of_steps=2000000, report_freq=1000)
plot_orbits(motions)

## Tribulations, Lessons Learned, and Conclusions

The biggest takeaway from this project for me, aside from the difficulties of modeling a Solar System through any
method, was that Jupyter just doesn't play nicely with a lot of animation suites.  To be fair, I know better and I
should have done my due diligence on the Turtle library in order to make sure it would work within Jupyter beforehand.
That was a huge setback for me, and it kept me from having one cohesive project that did everything I wanted it to.
I ended up instead with two projects, each of which did roughly half of what I was hoping to accomplish in total.

I'm extremely happy with how my Turtle Solar System Simulator turned out - I'm planning on building it out into a
larger project for an Open-Source course next term, and making it publicly available once it achieves a state of
integration with some of the more difficult orbital mechanics concepts (like n-body calculations).  But right now,
it's a neat toy that does a reasonable approximation of not one but two known Solar Systems.

I am not a physicist, nor am I a mathematician.  The n-body problem was a challenge; I experienced a great deal of
difficulty in understanding the math involved in the problem to the point that I could implement it, and coming up with
data structures that allowed for that implementation was similarly difficult.  However, once those hurdles had been
cleared, I didn't find the implementation itself so bad.  The attempts to find values which produced a specific result
were quite challenging; given time I'm sure that I could find values for unstable orbits.  But having put in almost
five hours of attempts across two different systems and not having come up with a result, I feel like I'm searching
for a needle in a haystack.

I can't help but feel a little disappointed with the overall results I was able to achieve, even though both portions
of the project were fine individually.  I really was hoping for something a little more cohesive, but time and
technology weren't necessarily on my side for this project.  I did enjoy working on it though, and I'm looking forward
to doing more modeling in the future!